In [ ]:
# default_exp data.utils

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
import matplotlib.pyplot as plt
import torch
from grade_classif.data.loaders import ImageLoader

In [ ]:
# export
def np_to_tensor(x, tensor_type):
    if tensor_type == 'image':
        x = x.transpose(2, 0, 1)
    x = torch.tensor(x)
    return x

In [ ]:
# export
def show_img(x, ax=None, figsize=(3,3), title=None, hide_axis=True, cmap='viridis', alpha=None, **kwargs):
    if ax is None: fig,ax = plt.subplots(figsize=figsize)
    xtr = dict(cmap=cmap, alpha=alpha, **kwargs)
    ax.imshow(x, **xtr)
    if hide_axis: ax.axis('off')
    if title: ax.set_title(title)
    return ax    

In [ ]:
# export
def load_batches(folder, bs=16, device='cpu'):
    x = []
    image_loader = ImageLoader('3G')
    for fn in folder.iterdir():
        if len(batch) < batch_size:
            img = image_loader(fn)
            img = np_to_tensor(img).to(device)
            x.append(img)
        else:
            yield torch.stack(x)
            x = []

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_train.ipynb.
Converted 02_predict.ipynb.
Converted 10_data_read.ipynb.
Converted 11_data_loaders.ipynb.
Converted 12_data_dataset.ipynb.
Converted 13_data_utils.ipynb.
Converted 14_data_transforms.ipynb.
Converted 20_models_pl_modules.ipynb.
Converted 21_models_modules.ipynb.
Converted 22_models_utils.ipynb.
Converted 23_models_hooks.ipynb.
Converted 80_params_defaults.ipynb.
Converted 81_params_norm.ipynb.
Converted 82_params_classif.ipynb.
Converted 83_params_predict.ipynb.
Converted 99_index.ipynb.
